<a href="https://colab.research.google.com/github/harrykeeran12/cs4040-miniproject/blob/main/CS4040_Research_Methods_Miniproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A zero-shot evaluation of small speech-to-text systems

The models evaluated are
- the small Fairseq model,
- the tiny Whisper model
- and the medium Conformer-CTC model,
due to their close parameter size.

A small subset of the dataset is loaded first.
Each step for the models is split up into:
- Loading the models.
- Translating the dataset into a form the model can understand using the processor.
- Running each model on the dataset, to check if they can infer the data correctly.

Then an evaluation is performed, with graphs.

**This will require the T4 GPU to be enabled on Google Colab.**


In [ ]:
# !huggingface-cli login
!pip3 install torch torchvision torchaudio datasets transformers google.colab  datasets[audio] evaluate
!pip3 install wget matplotlib>=3.3.2 text-unidecode

!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython packaging
!pip install "nemo_toolkit[all]"
!pip install nemo_toolkit['asr']

## Install NeMo
# BRANCH = 'main'
# !python -m pip install --upgrade --user git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

## Imports

In [ ]:
from torch.utils.data import DataLoader
from pprint import pprint
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration, WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset, IterableDataset, Audio
import pandas as pd
import numpy as np

import scipy.stats as stats
from evaluate import load
from IPython.display import display
import re
import matplotlib.pyplot as plt

import nemo.collections.asr as nemo_asr

## Load Dataset(Mozilla Common Voice v17)

In [ ]:
RANDOMSEED = 42
SAMPLE_NUMBER = 15
SAMPLING_RATE = 16000

cv_17: IterableDataset = load_dataset("mozilla-foundation/common_voice_17_0", "en", split="validated", streaming=True, trust_remote_code=True)

cv_17 = cv_17.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))

random10List : list[dict]= []
iterdataset = iter(cv_17.shuffle(seed=RANDOMSEED, buffer_size = SAMPLE_NUMBER))
for i in range(SAMPLE_NUMBER):
  audio = next(iterdataset)
  random10List.append(audio)
  # pprint(audio)

# pprint(random10List)

## Fairseq Small

In [ ]:
modelOne = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processorOne = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

fairseqOutput :dict[str,str]= {}
for audio in random10List:
  input_features = processorOne(
      audio["audio"]["array"],
      sampling_rate=SAMPLING_RATE,
      return_tensors="pt",
      padding=True
  ).input_features
  generated_ids = modelOne.generate(input_features=input_features)

  transcription = processorOne.batch_decode(generated_ids, skip_special_tokens=True)[0]
  fairseqOutput[audio["sentence"]] = transcription
  print(f'{audio["sentence"]} : {transcription}')

## Whisper Tiny

In [ ]:
# load model and processor
processorTwo = WhisperProcessor.from_pretrained("openai/whisper-tiny")
modelTwo = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
modelTwo.config.forced_decoder_ids = None


whisperOutput :dict[str,str]= {}
for audio in random10List:

  # Whisper requires specific chunking requirements of 30 seconds for each item.
  # The chunk size is represented by 30(seconds) times the sampling rate(16000)
  chunk_size = 30 * SAMPLING_RATE
  audioArray = audio["audio"]["array"]



  chunks = [audioArray[i:i + chunk_size] for i in range(0, len(audioArray), chunk_size)]
  # Pads out the chunks that may be too small.
  padded_chunks = [np.pad(chunk, (0, chunk_size - len(chunk)), 'constant', constant_values=0) for chunk in chunks if chunk.shape[0] < chunk_size]
  # NOTE: Chunking may result in word errors at the boundary of the word.

  transcriptions = []
  for chunk in padded_chunks:
    input_features = processorTwo(chunk, sampling_rate=SAMPLING_RATE,
    return_tensors="pt",padding=True).input_features
    predicted_ids = modelTwo.generate(input_features)
    transcription = processorTwo.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    transcriptions.append(transcription)
  # Joins each part of the chunked transcription together.
  full_transcription = " ".join(transcriptions)
  whisperOutput[audio["sentence"]] = full_transcription
  print(f'{audio["sentence"]} : {full_transcription}')


## Medium Conformer Model

In [ ]:
modelThree = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_medium")


CTCOutput :dict[str,str]= {}
for audio in random10List:
  full_transcription = modelThree.transcribe(audio=audio["audio"]["array"])[0]
  CTCOutput[audio["sentence"]] = full_transcription
  print(f'{audio["sentence"]} :  {full_transcription}')

## Evaluation:


In [ ]:
fairseqList = []
whisperList = []
sentenceList = []
CTCList = []

fairseqWERList = []
whisperWERList = []
CTCWERList = []

fairseqCERList = []
whisperCERList = []
CTCCERList = []

SIG_FIG = 4

# Loads the Word Error Rate Metric + the Character Error Rate Metric from HuggingFace.
wer = load("wer", module_type="metric")
cer = load("cer", module_type="metric")

for audio in random10List:
  # Gets the corresponding output for each sentence.
  sentence = audio["sentence"]

  filteredSentence = re.sub(r'[^\w\s]','',sentence).lower()

  fairseqTranscription = fairseqOutput[sentence].lower()
  whisperTranscription = whisperOutput[sentence].lower()
  CTCTranscription = CTCOutput[sentence].lower()
  # Computes the WER for each model.
  fairseqWER = wer.compute(predictions=[filteredSentence], references=[fairseqTranscription])
  whisperWER = wer.compute(predictions=[filteredSentence], references=[whisperTranscription])
  CTCWER = wer.compute(predictions=[filteredSentence], references=[CTCTranscription])

  fairseqCER = cer.compute(predictions=[filteredSentence], references=[fairseqTranscription])
  whisperCER = cer.compute(predictions=[filteredSentence], references=[whisperTranscription])
  CTCCER = cer.compute(predictions=[filteredSentence], references=[CTCTranscription])


  # Appends each sentence + transcribed sentence to a list to be added to the model later.
  sentenceList.append(filteredSentence)
  fairseqList.append(fairseqTranscription)
  whisperList.append(whisperTranscription)
  CTCList.append(CTCTranscription)
  # Appends the WER to a list + rounds the number.
  fairseqWERList.append(round(fairseqWER, SIG_FIG))
  whisperWERList.append(round(whisperWER, SIG_FIG))
  CTCWERList.append(round(CTCWER, SIG_FIG))

  # Appends the CER to a list + rounds the number.
  fairseqCERList.append(round(fairseqCER, SIG_FIG))
  whisperCERList.append(round(whisperCER, SIG_FIG))
  CTCCERList.append(round(CTCCER, SIG_FIG))


transcriptionDF = pd.DataFrame({"Sentence(after preprocessing)": sentenceList, "Fairseq Transcription": fairseqList, "Whisper Transcription": whisperList, "CTC-Conformer Transcription" : CTCList})

WERDF = pd.DataFrame({"Sentence(after preprocessing)": sentenceList, "Fairseq WER": fairseqWERList,"Whisper WER" : whisperWERList, "CTC-Conformer WER": CTCWERList})

CERDF = pd.DataFrame({"Sentence(after preprocessing)": sentenceList, "Fairseq CER": fairseqCERList,"Whisper CER" : whisperCERList, "CTC-Conformer CER": CTCCERList})

display(WERDF)
display(CERDF)

### WER for each sentence

In [ ]:
plt.figure(figsize=(10, 6))

# Create bar plots for each model's WER
plt.bar(WERDF.index, WERDF["Fairseq WER"], label="Fairseq", width=0.25)
plt.bar(WERDF.index + 0.25, WERDF["Whisper WER"], label="Whisper", width=0.25)
plt.bar(WERDF.index + 0.5, WERDF["CTC-Conformer WER"], label="CTC-Conformer", width=0.25)


plt.xlabel("Sentence Number")
plt.ylabel("WER")
plt.title("WER Rates for Each Sentence")
plt.xticks(WERDF.index + 0.25, WERDF.index) # adjust x-axis ticks
plt.legend()
plt.tight_layout()
plt.ylim(0, 1)
plt.show()

### CER for each sentence

In [ ]:
plt.figure(figsize=(10, 6))

# Create bar plots for each model's CER
plt.bar(CERDF.index, CERDF["Fairseq CER"], label="Fairseq", width=0.25)
plt.bar(CERDF.index + 0.25, CERDF["Whisper CER"], label="Whisper", width=0.25)
plt.bar(CERDF.index + 0.5, CERDF["CTC-Conformer CER"], label="CTC-Conformer", width=0.25)


plt.xlabel("Sentence Number")
plt.ylabel("CER")
plt.title("CER Rates for Each Sentence")
plt.xticks(CERDF.index + 0.25, CERDF.index) # adjust x-axis ticks
plt.ylim(0, 1)
plt.legend()
plt.tight_layout()
plt.show()

### Median WER + CER for each model

In [ ]:
print("WER means: \n")
print(WERDF["Fairseq WER"].mean())
print(WERDF["Whisper WER"].mean())
print(WERDF["CTC-Conformer WER"].mean())
print("WER ranges: \n")
print(f'{WERDF["Fairseq WER"].min()}, {WERDF["Fairseq WER"].max()}')
print(f'{WERDF["Whisper WER"].min()}, {WERDF["Whisper WER"].max()}')
print(f'{WERDF["CTC-Conformer WER"].min()}, {WERDF["CTC-Conformer WER"].max()}')
print("CER means: \n")
print(CERDF["Fairseq CER"].mean())
print(CERDF["Whisper CER"].mean())
print(CERDF["CTC-Conformer CER"].mean())
print("CER ranges: \n")
print(CERDF["Fairseq CER"].min(), CERDF["Fairseq CER"].max())
print(CERDF["Whisper CER"].min(), CERDF["Whisper CER"].max())
print(CERDF["CTC-Conformer CER"].min(), CERDF["CTC-Conformer CER"].max())

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 12))

models = ["Fairseq", "Whisper", "CTC-Conformer"]
WERmedians = [WERDF["Fairseq WER"].median(), WERDF["Whisper WER"].median(), WERDF["CTC-Conformer WER"].median()]
CERmedians = [CERDF["Fairseq CER"].median(), CERDF["Whisper CER"].median(), CERDF["CTC-Conformer CER"].median()]
colors = ['tab:blue', 'tab:orange', 'tab:green']

ax[0].bar(models, WERmedians, color=colors)
ax[0].set_xlabel("Median Name")
ax[0].set_ylabel("Median WER(lowest is better)")
ax[0].set_title("Median WER per Model")
# ax[0].set_ylim(0, 0.5)

ax[1].bar(models, CERmedians, color=colors)
ax[1].set_xlabel("Model Name")
ax[1].set_ylabel("Median CER(lowest is better)")
ax[1].set_title("Median CER per Model")
# ax[1].set_ylim(0, 0.5)

plt.tight_layout()

plt.show()

## Statistical Analysis


In [ ]:
# Combine WER data for all models
wer_data = [WERDF["Fairseq WER"], WERDF["Whisper WER"], WERDF["CTC-Conformer WER"]]

# Perform ANOVA test
f_statistic, p_value = stats.f_oneway(*wer_data)

print(f"ANOVA test for WER:")
print(f"F-statistic: {f_statistic}")
print(f"P-value: {p_value}")

# Combine CER data for all models
cer_data = [CERDF["Fairseq CER"], CERDF["Whisper CER"], CERDF["CTC-Conformer CER"]]

# Perform ANOVA test
f_statistic, p_value = stats.f_oneway(*cer_data)

print(f"\nANOVA test for CER:")
print(f"F-statistic: {f_statistic}")
print(f"P-value: {p_value}")


ANOVA test for WER:
F-statistic: 3.932834919478482
P-value: 0.027189442100556207

ANOVA test for CER:
F-statistic: 2.9251679964762847
P-value: 0.06466216986778027
